In [1]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta

sensor_dict = {'M_M_Vrms': [0.0004, 0.0041], 'M_M_Arms': [0.9, 2.7], 'M_M_Temp': [84.56, 86.9], 'Cl_Vrms': [0.0001, 0.0083], 'Cl_Arms': [0.2, 7.2], 'Cl_Temp': [82.76, 83.84], 'F_L_Pin_Vrms': [0.0001, 0.0041], 'F_L_Pin_Arms': [0.1, 6.7], 'F_L_Pin_Temp': [85.64, 88.88], 'F_R_Pin_Vrms': [0.0001, 0.0037], 'F_R_Pin_Arms': [0.2, 5.9], 'F_R_Pin_Temp': [85.28, 88.7], 'F_L_B_Vrms': [0.0001, 0.0022], 'F_L_B_Arms': [0.1, 5.7], 'F_L_B_Temp': [98.78001, 102.38], 'F_R_B_Vrms': [0.0001, 0.002], 'F_R_B_Arms': [0.1, 5.9], 'F_R_B_Temp': [98.24, 102.38], 'R_L_Pin_Vrms': [0.0001, 0.0038], 'R_L_Pin_Arms': [0.2, 4.3], 'R_L_Pin_Temp': [87.26, 88.52], 'R_R_Pin_Vrms': [0.0001, 0.0035], 'R_R_Pin_Arms': [0.2, 4.9], 'R_R_Pin_Temp': [87.98, 89.6], 'R_L_B_Vrms': [0.0001, 0.0029], 'R_L_B_Arms': [0.1, 5.7], 'R_L_B_Temp': [94.46001, 97.52], 'R_R_B_Vrms': [0.0001, 0.0024], 'R_R_B_Arms': [0.1, 4.9], 'R_R_B_Temp': [93.74, 97.16], 'Press_angle': [4.570313, 195.1172], 'Ram_Adjust_Chain_On/Off': [0.0, 0.0], 'F_L_Nut_Vrms': [0.0, 0.0038], 'F_L_Nut_Arms': [0.1, 3.0], 'F_L_Nut_Temp': [84.38, 85.82], 'F_R_Nut_Vrms': [0.0, 0.0038], 'F_R_Nut_Arms': [0.1, 3.2], 'F_R_Nut_Temp': [82.04, 82.94], 'R_L_Nut_Vrms': [0.0, 0.0034], 'R_L_Nut_Arms': [0.0, 2.2], 'R_L_Nut_Temp': [83.3, 84.92], 'R_R_Nut_Vrms': [0.0, 0.0034], 'R_R_Nut_Arms': [0.1, 2.7], 'R_R_Nut_Temp': [83.48, 85.28], 'Ram_Adjust_Vrms': [0.0002, 0.0146], 'Ram_Adjust_Arms': [0.2, 9.0], 'Ram_Adjust_Temp': [83.84, 85.46001], 'F_R_Gibbs_Vrms': [0.0, 0.0026], 'F_R_Gibbs_Arms': [0.0, 2.4], 'F_R_Gibbs_Temp': [32.0, 80.96001], 'R_R_Gibbs_Vrms': [0.0, 0.0025], 'R_R_Gibbs_Arms': [0.0, 3.0], 'R_R_Gibbs_Temp': [32.0, 82.22], 'F_L_Gibbs_Vrms': [0.0, 0.0032], 'F_L_Gibbs_Arms': [0.0, 2.9], 'F_L_Gibbs_Temp': [32.0, 82.4], 'R_L_Gibbs_Vrms': [0.0, 0.0023], 'R_L_Gibbs_Arms': [0.0, 2.4], 'R_L_Gibbs_Temp': [32.0, 81.5]}

In [2]:
class Error:
    def __init__(self,sensor_name,description):
        self.sensor_name = sensor_name
        self.description = description

    def __repr__(self):
        return f"{self.sensor_name}: {self.description}"
    
def press_11_alert_generator():
    press = "Press_11"

    myclient_global = pymongo.MongoClient(host = "128.121.34.13", connect = True )
    press_db = myclient_global[press]
    end = datetime.now()
    start   = end - timedelta(minutes=60)

    min_max = pd.DataFrame()

    for batch in ['BATCH_1','BATCH_2']:
        collection= press_db[batch]
        projection = {}
        projection['_id'] = 0
        projection['Date'] = 1

        for sensor in sensor_dict.keys():
            projection[sensor] = 1

        QUERY = {"Date": {'$gte': start, '$lt':  end}}
        results = collection.find(QUERY,projection)
        df1 = pd.DataFrame(results).set_index('Date')
        describe = df1.describe().T
        describe = describe[['min','max']]
        min_max = pd.concat([min_max,describe],axis=0)

    print(min_max)

    error_list = []
    for i in range(len(min_max)):
        name = min_max.iloc[i].name
        lower = min_max.iloc[i].min()
        upper = min_max.iloc[i].max()
        [lower_limit, upper_limit] = sensor_dict[name]

        if lower < lower_limit and upper > upper_limit:
            lower_error = True
            upper_error = True
        elif lower < lower_limit:
            lower_error = True
            upper_error = False
        elif upper > upper_limit:
            lower_error = False
            upper_error = True
        else:
            lower_error = False
            upper_error = False

        if lower_error or upper_error:
            descr = ""
            if lower_error and upper_error:
                descr = "Sensor registered data both below limit and above upper limit in the last 5 minutes"
            elif lower_error:
                descr = "Sensor registered data below lower limit"
            elif upper_error:
                descr = "Sensor registered data above upper limit"
            error = Error(name,descr)
            error_list.append(error)

    return error_list

                       min          max
F_L_Nut_Vrms       0.00000     0.005500
F_L_Nut_Arms       0.00000     7.400000
F_L_Nut_Temp      93.92000    95.539990
F_R_Nut_Vrms       0.00000     0.005600
F_R_Nut_Arms       0.00000     9.400001
F_R_Nut_Temp      91.76000    93.020000
R_L_Nut_Vrms       0.00000     0.005400
R_L_Nut_Arms       0.00000     5.600000
R_L_Nut_Temp      93.02000    94.460010
R_R_Nut_Vrms       0.00000     0.005300
R_R_Nut_Arms       0.00000     6.500000
R_R_Nut_Temp      93.02000    94.460010
Ram_Adjust_Vrms    0.00000     0.020200
Ram_Adjust_Arms    0.10000     9.700000
Ram_Adjust_Temp   93.74000    95.539990
M_M_Vrms           0.00000     0.011100
M_M_Arms           0.10000    30.400000
M_M_Temp          95.36000   100.940000
Cl_Vrms            0.00000     0.010400
Cl_Arms            0.10000    17.300000
Cl_Temp           91.58000    95.900000
F_L_Pin_Vrms       0.00000     0.008100
F_L_Pin_Arms       0.10000    12.300000
F_L_Pin_Temp      94.28000    95.360000
